[] 3.1 Use gensim.downloader.load to load the word2vec-google-news-300 pretrained embedding model.

In [9]:
import gensim.downloader

word2vec_googlenews = gensim.downloader.load('word2vec-google-news-300')



3.2 Use the tokenizer from nltk to extract words from the Reddit posts. Display the number of tokens in the training set.


In [11]:
import json
import gzip
import nltk

# jsonfiledirectory = "/Users/rosiers/Documents/GitHub/472-assignment1/goemotions.json.gz"
jsonfiledirectory = "C:\\Users\\Krish\\.vscode\\472-assignment1\\goemotions.json.gz"

with gzip.open(jsonfiledirectory, "r") as f:
    data = json.loads(f.read().decode("utf-8"))
strings_token = []
for item in data:
    strings_token.append(nltk.word_tokenize(item[0]))
count = 0
for str in strings_token:
    count = count + len(str)
print(count)

2642128


3.3 Compute the embedding of a Reddit post as the average of the embeddings of its words. If a word has no embedding in Word2Vec, skip it.

In [12]:
# create an empty list to add each average embedding to
embeddings = []
#strings_token is the tokenized sentences
for s in strings_token:
    #for each sentence create a vector to hold the embeddings for each token in the string
    # it will be wiped each time
    strings_embedding = []

    #for each token in the sentence check if it has embedding and add it to the list or skip
    for tkn in s:
        if tkn in word2vec_googlenews:
            strings_embedding.append(word2vec_googlenews[tkn])
        else:
            continue

    #use zip to put all elements from each embedding into list of the same index
    embd = zip(*strings_embedding)
    #calculate the average by finding the sum of the elements at each index, and dividing it by the number of elements
    average_embedding = []
    for e in embd:
        avg = sum(e)/len(e)
        average_embedding.append(avg)
    #once all the averages are computed, append the average embedding as an element
    embeddings.append(average_embedding)
print(embeddings[1])

[0.0625732421875, 0.0309539794921875, 0.0525146484375, 0.18604736328125, -0.09674072265625, 0.0177734375, 0.0715301513671875, -0.0430419921875, -0.009185791015625, -0.009529876708984374, -0.121923828125, -0.18212890625, -0.0642822265625, 0.03070068359375, -0.1669189453125, 0.1212158203125, 0.067041015625, 0.095947265625, 0.0786376953125, -0.1344970703125, -0.0189697265625, 0.01287841796875, 0.184210205078125, 0.0110565185546875, 0.02649974822998047, 0.051744842529296876, -0.115362548828125, 0.043670654296875, 0.01415557861328125, -0.053167724609375, -0.068359375, 0.14326171875, -0.038232421875, -0.06484375, -0.10544891357421875, 0.04044189453125, 0.087939453125, 0.051953125, 0.000640869140625, 0.11961212158203124, 0.04735107421875, -0.04951171875, 0.259423828125, -0.0633392333984375, -0.01748046875, -0.023974609375, 0.02947998046875, -0.10185546875, 0.0018310546875, -0.01885986328125, -0.09895172119140624, 0.0999755859375, -0.00535888671875, 0.06209716796875, 0.0124420166015625, -0.002

3.4 Compute and display the overall hit rates of the training and test sets (i.e. the % of words in the Reddit posts for which an embedding is found in Word2Vec).

In [18]:
counter = 0

for i in strings_token: 
    for tkn in i:
        if tkn in word2vec_googlenews:
            counter= counter+1
        else:
            continue

print("Overall hitrates:")
hitrate= counter/count
print(hitrate)

Overall hitrates:
0.7745063827339175


3.5 Train a Base-MLP: a Multi-Layered Perceptron (neural network.MLPClassifier) with the default parameters.

In [19]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
import numpy

posts = []
sentiment = []
emotions = []
for item in data:
    posts.append(item[0])
    sentiment.append(item[2])
    emotions.append(item[1])

X_train, X_test, y_train, y_test = train_test_split(embeddings, sentiment, test_size=0.2)

print(X_train[0])
print(y_train[0])
#x_numpy = numpy.array(X_train).reshape(-1,1)
#y_numpy = numpy.array(y_train).reshape(-1,1)
basemlp = MLPClassifier(max_iter=4)
basemlp.fit(X_train, y_train)
print(basemlp.score(X_test,y_test))

[-0.028157552083333332, 0.19417317708333334, -0.033040364583333336, 0.13557942708333334, -0.06819661458333333, 0.09309895833333333, 0.04833984375, 0.010498046875, 0.053466796875, 0.10432942708333333, -0.010904947916666666, -0.171875, -0.10477701822916667, -0.0126953125, -0.16227213541666666, 0.055501302083333336, 0.07666015625, 0.1513671875, -0.16288248697916666, -0.25439453125, -0.19590250651041666, 0.07405598958333333, 0.19184048970540366, -0.01556396484375, 0.072021484375, 0.21044921875, -0.15681966145833334, 0.046915690104166664, 0.11165364583333333, -0.03656005859375, -0.021158854166666668, 0.0087890625, 0.0947265625, -0.17431640625, -0.11397298177083333, 0.10872395833333333, -0.1049041748046875, 0.1746826171875, -0.04638671875, 0.14206949869791666, 0.17154947916666666, -0.03759765625, 0.16300455729166666, -0.09090169270833333, 0.05224609375, -0.06368001302083333, -0.041097005208333336, -0.018239339192708332, -0.0052490234375, 0.0535888671875, -0.2333984375, 0.13704427083333334, 0

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (137456,) + inhomogeneous part.

3.6 Train a Top-MLP: a better performing Multi-Layered Perceptron found with whatever hyperparameters you want.

In [10]:
from sklearn.model_selection import GridSearchCV
topmlp = MLPClassifier(max_iter=1)

param_grid = {
    'activation' : ['sigmoid', 'tanh', 'relu', 'identity'],
    'hidden_layer_sizes': [(10,10,10), (30,50)],
    'solver': ['sgd', 'adam']
}
mlp_grid = GridSearchCV(topmlp,param_grid,n_jobs=-1)
mlp_grid.fit(X_train, y_train)
print("TOP-MLP: ")
print(mlp_grid.score(X_test, y_test))


/Users/rosiers/Library/jupyterlab-desktop/jlab_server/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


KeyboardInterrupt: 

3.7  Display the performance of your classifiers using metrics.classification report and add these to your performance file.